In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [114]:
df = pd.read_csv("/work/for_qiita/dataset/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [76]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [115]:
def quantify_sex(sex):
    if sex == 'male':
        return 0
    elif sex == 'female':
        return 1
    else:
        return 0.5
def quantify_embarked(embarked):
    if embarked == 'C':
        return 0
    elif embarked == 'Q':
        return 1
    elif embarked == 'S':
        return 2
    else:
        return 2

In [116]:
df["Sex"] = df["Sex"].apply(quantify_sex)
df["Embarked"] = df["Embarked"].apply(quantify_embarked)
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Companions"] = df.apply(lambda x : x["SibSp"] + x["Parch"] + 1, axis=1)

In [117]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Companions
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0,2
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2,1


In [121]:
df = df[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Companions"]]
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Companions
0,0,3,0,22.0,1,0,7.2500,2,2
1,1,1,1,38.0,1,0,71.2833,0,2
2,1,3,1,26.0,0,0,7.9250,2,1
3,1,1,1,35.0,1,0,53.1000,2,2
4,0,3,0,35.0,0,0,8.0500,2,1


In [133]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df_scaler = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [134]:
df_scaler = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
df_scaler.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,2
1,1,1,38.0,1,0,71.2833,0
2,3,1,26.0,0,0,7.9250,2
3,1,1,35.0,1,0,53.1000,2
4,3,0,35.0,0,0,8.0500,2


In [128]:
from sklearn.decomposition import FactorAnalysis as FA
n_components=3
fa = FA(n_components, max_iter=500)
fitted = fa.fit_transform(df_scaler)

In [129]:
fa.components_.T

array([[ 2.16020573e-02,  1.09445386e-01,  4.06404652e-01],
       [ 6.35165090e-02, -5.63759961e-02, -8.48155140e-01],
       [ 2.07178699e-01,  1.32436730e-01,  2.25831876e-01],
       [-2.43926594e-01,  4.29996300e-02,  3.03266180e-01],
       [ 8.69281448e-01, -4.94317474e-01,  2.91669222e-13],
       [ 8.10387923e-01,  5.85893689e-01, -3.76705337e-13],
       [ 2.20269606e-01,  6.43821852e-02,  6.55194045e-01],
       [ 6.55555931e-02, -2.27465717e-02, -2.35057908e-01],
       [ 9.98980391e-01, -4.51461918e-02,  1.10691265e-14]])

In [130]:
Factor_loading_matrix = fa.components_.T
pd.DataFrame(Factor_loading_matrix, columns=["factor_1", "factor_2", 
                                            "factor_3"], index=[df.columns])

,factor_1,factor_2,factor_3
Survived,0.021602,0.109445,4.064047e-01
Pclass,0.063517,-0.056376,-8.481551e-01
Sex,0.207179,0.132437,2.258319e-01
Age,-0.243927,0.043000,3.032662e-01
SibSp,0.869281,-0.494317,2.916692e-13
Parch,0.810388,0.585894,-3.767053e-13
Fare,0.220270,0.064382,6.551940e-01
Embarked,0.065556,-0.022747,-2.350579e-01
Companions,0.998980,-0.045146,1.106913e-14


In [135]:
df = pd.read_csv("/work/for_kikagaku/wine_class.csv")
df.head()

,Class,Alcohol,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Color intensity,Hue,Proline
0,1,14.23,2.43,15.6,127,2.80,3.06,0.28,5.64,1.04,1065
1,1,13.20,2.14,11.2,100,2.65,2.76,0.26,4.38,1.05,1050
2,1,13.16,2.67,18.6,101,2.80,3.24,0.30,5.68,1.03,1185
3,1,14.37,2.50,16.8,113,3.85,3.49,0.24,7.80,0.86,1480
4,1,13.24,2.87,21.0,118,2.80,2.69,0.39,4.32,1.04,735


In [136]:
df.isnull().sum()

Class                   0
Alcohol                 0
Ash                     0
Alcalinity of ash       0
Magnesium               0
Total phenols           0
Flavanoids              0
Nonflavanoid phenols    0
Color intensity         0
Hue                     0
Proline                 0
dtype: int64

In [137]:
df_scaler = scaler.fit_transform(df)

In [141]:
df_scaler = pd.DataFrame(df_scaler, columns=df.columns)
df_scaler.head()

,Class,Alcohol,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Color intensity,Hue,Proline
0,-1.213944,1.518613,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,0.251717,0.362177,1.013009
1,-1.213944,0.246290,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.293321,0.406051,0.965242
2,-1.213944,0.196879,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,0.269020,0.318304,1.395148
3,-1.213944,1.691550,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.186068,-0.427544,2.334574
4,-1.213944,0.295700,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,-0.319276,0.362177,-0.037874


In [145]:
from sklearn.datasets import load_wine
wine = load_wine()
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [146]:
print(wine.target_names)
print(wine.feature_names)

['class_0' 'class_1' 'class_2']
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
